In [ ]:
%cd /content
import IPython.display
import os
import sys

if not os.path.exists("/content/sd-inference-server"):
    !git clone https://github.com/arenasys/sd-inference-server.git

%cd /content/sd-inference-server
model_folder = "/content/sd-inference-server/models"
try:
    # decline the popup to use the local folder ^
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    model_folder = "/content/drive/My Drive/qDiffusion/models"
    if not os.path.exists(model_folder):
        !mkdir '/content/drive/My Drive/qDiffusion' -p
        !cp -r 'models/' '/content/drive/My Drive/qDiffusion/'
except Exception as e:
    pass

if not os.path.exists("venv"):
    !curl -OJL https://huggingface.co/datasets/arenasys/qDiffusion/resolve/main/cached_venv5.tar
    !tar xf cached_venv5.tar; mv cached_venv5 venv; rm cached_venv5.tar
    !rm /content/sd-inference-server/venv/lib/python3.10/site-packages/pathlib.py
    !pip install timm==0.6.13
    !pip install tomesd==0.1.3
    !pip install torchmetrics
    !pip install git+https://github.com/facebookresearch/segment-anything.git
    !pip uninstall -y tensorflow

    !wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
    !wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
    !wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
    !wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
    !apt install -qq libunwind8-dev
    !dpkg -i *.deb
    %env LD_PRELOAD=libtcmalloc.so

    IPython.display.clear_output()

if not sys.path[0] == "/content/sd-inference-server/":
    sys.path.insert(0, "/content/sd-inference-server/venv/lib/python3.10/site-packages")
    sys.path.insert(0, "/content/sd-inference-server/")

from pycloudflared import try_cloudflare
tunnel_url = try_cloudflare(port=28888, verbose=False)
print("ENDPOINT:", tunnel_url.tunnel.replace("https", "wss"))

!python remote.py "$model_folder"

try_cloudflare.terminate(28888)